In [1]:
import librosa
import scipy.signal as signal
import numpy as np
import pandas as pd
import joblib

In [2]:
import os
import multiprocessing
from multiprocessing import Pool
from functools import partial
import numpy as np
import pandas as pd 
from data_loader_v2 import data_loader_v2

In [5]:
#환경 확인
import pyaudio
import wave
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 20
WAVE_OUTPUT_FILENAME = "test_file.wav"
audio = pyaudio.PyAudio()

In [6]:
# start Recording
stream = audio.open(format=pyaudio.paInt16,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    input_device_index=1,
                    frames_per_buffer=CHUNK)
print ("recording...")
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print ("finished recording")

recording...
finished recording


In [7]:
stream.stop_stream()
stream.close()
audio.terminate()
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

In [19]:
#test_file.wav
audio_sample, sampling_rate = librosa.load("test_file.wav", sr = None)

#stft 처리
#변수는 임의 상정(혹은 기본값)
S = np.abs(librosa.stft(audio_sample, n_fft=1024, hop_length=512, win_length = 1024, window=signal.hann))
pitches, magnitudes = librosa.piptrack(S=S, sr=sampling_rate, fmin=400, fmax=3000)
shape = np.shape(pitches)
nb_pitch = shape[0]
nb_time = shape[1]
#피치 생성
for i in range(0, nb_time):
    index = magnitudes[:,i].argmax()
    pitch = pitches[index,i]
df_pitch = pd.DataFrame(pitches)
#데이터 생성
#col : 피쳐(피치값), row : time
df_pitch = df_pitch.T
df_pitch.to_csv("data/test/1.csv", index=False)
    
#편의를 위해 추가한 코드...
#col 크기 94로 통일
df_tmp = pd.read_csv("data/test/1.csv")
drop_col = [str(i) for i in range(95, nb_pitch)]
df_tmp = df_tmp.drop(drop_col, axis=1)
df_tmp.to_csv("data/test/1.csv")

In [20]:
# 563줄만 가져오기(녹음할때도 이거 기준으로 잘라야함)
def data_loader_all(func, files, folder='', train_label=None, event_time=0, nrows=563):   
    #변수 값 고정하여 파생 함수
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    
    #cpu에 작업 분할
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    
    #df생성
    combined_df = pd.concat(df_list)    
    return combined_df

In [21]:
#데이터 불러오기
test_folder = 'data/test/'
test_list = os.listdir(test_folder)
#test_label = pd.read_csv(test_label_path, index_col=0)
test = data_loader_all(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=0, nrows=563)

In [22]:
model = joblib.load('model_lstm1.pkl')
de_label = pd.read_csv('de_train_label.csv', engine='python', index_col = None)

In [23]:
test = test.values
test_t = test.reshape(test.shape[0], 95, 1)
pred = model.predict_proba(test_t)
pred = pred.reshape(563, int(pred.shape[0]/563))

In [24]:
#평균값
def ans(nparray):
    tmp_list=[]
    for i in range (0, nparray.shape[1]):
        tmp=0
        for j in range (0, nparray.shape[0]):
            tmp+=nparray[j][i]
        tmp_list.append(round(tmp/563))
    return tmp_list

In [25]:
ans_list = ans(pred)
for i in range (0, len(ans_list)):
    print("이번 역은 "+de_label['name'][ans_list[i]]+"역 입니다.")

이번 역은 대림역 입니다.
